In [ ]:
import pandas as pd
import re
import demoji
import unidecode
import numpy as np

In [ ]:
data = pd.read_csv("data/dataTweeter.csv", sep=";", encoding='utf8')

In [ ]:
def cleanText(tweet):
    if "|" in tweet:
        tweet = tweet.split("|")[1]
    
    tweet = ' '.join(re.sub(r"http\S+", "", tweet).split())
    tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)|([0-9])", " ", tweet).split())
    tweet = ' '.join(re.sub("(\w+:\/\/\S+)", " ", tweet).split())
    tweet = ' '.join(re.sub("[\_\|\.\,\"\'\!\?\:\;\$\-\(\)\=]", " ", tweet).split())
    tweet = tweet.lower()
    
    le = list(demoji.findall(tweet))
    for i in le:
        tweet = tweet.replace(i, "")
    
    if tweet.startswith('rt '):
        tweet = tweet.replace("rt ", "")
    
    lNewTweet = []
    for i in tweet.split(" "):
        lNewTweet.append(i)
    
    newTweet = " ".join(lNewTweet)
        
    return unidecode.unidecode(newTweet.replace(" rt ", ""))

In [ ]:
data["tweet_text_clean"] = data.POST.apply(cleanText)
data.drop(columns=["ID", "USER", "POST"], inplace=True)
data.head()

In [ ]:
token_ = [doc.split(" ") for doc in data["tweet_text_clean"].tolist()]

In [ ]:
token_3 = []
[[token_3.append(y) for y in x] for x in token_]

In [48]:
def getDataInNumbers(initialTxt, limit=4):
    _dict = {}
   
    for i in range(len(initialTxt)-limit):
        
        x = initialTxt[i:i+limit]
        y = initialTxt[i+limit]
        
        if x not in _dict:
            _dict[x] = {}
            _dict[x][y] = 1
        else:
            if y not in _dict[x]:
                _dict[x][y] = 1
            else:
                _dict[x][y] += 1
    
    return _dict

In [49]:
def getProb(_dict):
    for x_key in _dict.keys():
        _sum = float(sum(_dict[x_key].values()))
        for y_key in _dict[x_key].keys():
            _dict[x_key][y_key] = _dict[x_key][y_key]/_sum
    
    return _dict
        

In [74]:
def predictNextText(initial, mChains, limit=4):   
    
    initial = initial[-limit:]
 
    if initial not in mChains:
        return " "    
    
    chars = list(mChains[initial].keys())
    probs = list(mChains[initial].values())
    
    probChoice = np.random.choice(chars, p=probs) 
    
    return probChoice

In [75]:
mChains = getProb(getDataInNumbers(" ".join(token_3)))

In [79]:
import time
s = "Brasil"

for i in range(0, 500):
    time.sleep(0.2)
    
    s+= predictNextText(s, mChains)
    print (s, end="\r")
    
    